In [ ]:
'''
DCB unrestricted Hartree-Fock for CompChem exercises
'''

import numpy as np
import scipy as sp
from pyscf import gto, scf, ao2mo
from functools import reduce

#tell numpy how to print arrays
np.set_printoptions(formatter={'float': '{: 0.6f}'.format})

#define molecule
mol = gto.M()

mol.atom =("""
H 0 0 -0.37
H 0 0  0.37
""")
mol.spin = 0
mol.basis = '6-31G'
mol.build()
Norb = mol.nao_nr()
print("There are %d basis functions and %d electrons\n" % (Norb, mol.nelectron))

#convergence criteria
e_conv = 1.e-8
#SCF cycle details
damp_value = 0.20
damp_start = 100
ncycl = 100
print_matrices = True

#distribute electrons into alpha and beta channels
if (mol.nelectron %2 != 0):
    nel_a = int(0.5 * (mol.nelectron + 1))
    nel_b = int(0.5 * (mol.nelectron - 1))
else:
    nel_a = int(0.5 * mol.nelectron)
    nel_b = nel_a
    
#helper routine to nicely print a matrix
def print_matrix(label, matrix):
    if print_matrices:
        print(label)
        print(matrix)
        print()
    
#get one-electron matrix elements for core hamiltonian
T = mol.intor_symmetric('cint1e_kin_sph')
V = mol.intor_symmetric('cint1e_nuc_sph')
H = T + V

print_matrix("Kinetic energy matrix", T)
print_matrix("Nuclear interaction matrix", V)
print_matrix("Core Hamiltonian matrix", H)

#get overlap matrix
S = mol.intor_symmetric('cint1e_ovlp_sph')
print_matrix("Overlap matrix", S)

#get two-electron integrals
g = mol.intor('cint2e_sph')

#get the orthogonalisation matrix
X = sp.linalg.fractional_matrix_power(S, -0.5)
print_matrix("Orthogonalisation matrix", X)
print_matrix("X^TSX (orthogonalisation check)", reduce(np.dot, (X.T, S, X)))

#initialise density matrices to zero (see lecture)
P_a = np.zeros((Norb, Norb))
P_b = np.zeros((Norb, Norb))
P = P_a + P_b
print_matrix("Initial alpha density matrix", P_a)
print_matrix("Initial beta density matrix", P_a)
print_matrix("Initial total density matrix", P_a)

#SCF procedure
E_old = 0.0
Fa_old = None
Fb_old = None
maxcyc = None
print("------ Start SCF cycle ------")
for iteration in range(ncycl):

    #use density matrix to build elements of Fock matrix
    J = np.einsum("pqrs,rs->pq", g, P)
    K_a = np.einsum("prqs,rs->pq", g, P_a)
    K_b = np.einsum("prqs,rs->pq", g, P_b)

    #build Fock matrix
    print("An evil programmer forgot to finish the code")
    raise
    #you will have to change the zeros in the following lines to something more useful
    #and then delete the print and raise lines above
    F_a = 0
    F_b = 0
    print_matrix("Alpha Fock matrix before damping", F_a)
    print_matrix("Beta Fock matrix before damping", F_b)

    #see if we should damp (mix in old density) to improve convergence
    if iteration >= damp_start:
        F_a = damp_value * Fa_old + (1.0 - damp_value) * F_a
        F_b = damp_value * Fb_old + (1.0 - damp_value) * F_b

    #store the Fock matrix for damping in the next iteration
    Fa_old = F_a
    Fb_old = F_b
    print_matrix("Alpha Fock matrix after damping", F_a)
    print_matrix("Beta Fock matrix after damping", F_b)
    
    #solve the FC = SCe equation by use of a transformation matrix X
    #compute F'
    Fp_a = reduce(np.dot, (X.T, F_a, X))
    Fp_b = reduce(np.dot, (X.T, F_b, X))
    #diagonalise F' and get C' and e(psilon)
    eps_a, Cp_a = np.linalg.eigh(Fp_a)
    eps_b, Cp_b = np.linalg.eigh(Fp_b)
    #transform back to get C
    C_a = X.dot(Cp_a)
    C_b = X.dot(Cp_b)
    print_matrix("Alpha coefficients", C_a)
    print_matrix("Beta coefficients", C_b)

    #extract the occupied subspace of the coefficient matrices
    Cocc_a, Cocc_b = C_a[:, :nel_a], C_b[:, :nel_b]
    print_matrix("Occupied alpha coefficients", Cocc_a)
    print_matrix("Occupied beta coefficients", Cocc_b)

    #build new density matrix
    P_a = Cocc_a.dot(Cocc_a.T)
    P_b = Cocc_b.dot(Cocc_b.T)
    P = P_a + P_b
    print_matrix("Alpha density matrix", P_a)
    print_matrix("Beta density matrix", P_b)
    print_matrix("Total density matrix", P)

    # Compute the energy and difference from last iteration
    E_electric = 0.5 * np.sum(H * P + P_a * F_a + P_b * F_b)
    E_total = E_electric + mol.energy_nuc()
    E_diff = E_total - E_old
    E_old = E_total
    
    #print some output
    print("Iter=%3d  E = % 16.12f  E_diff = % 8.4e  Damp = %s" %
            (iteration, E_total, E_diff, iteration >= damp_start))

    #stop if energy has converged
    if (abs(E_diff) < e_conv):
        maxcyc = iteration
        break

    #if in last cycle set flag that we have not converged
    if (iteration == ncycl - 1):
        maxcyc = ncycl

#analyze if SCF has converged or ran into ncycl limit
if (maxcyc == ncycl):
    print("SCF has not finished!\n")
else:
    print("SCF has finished!\n")

    #perform Mulliken population
    print("Number of electrons = ", np.trace(P @ S))
    labels = mol.ao_labels()
    mull_a = np.diagonal(P_a @ S)
    mull_b = np.diagonal(P_b @ S)
    mull = np.diagonal(P @ S)
    qat_a = np.zeros(mol.natm)
    qat_b = np.zeros(mol.natm)
    qat = np.zeros(mol.natm)
    print("AO charges")
    print("AO label   | alpha    | beta     | total")
    for i in range(len(labels)):
        print("%10s | %.6f | %.6f | %.6f" % (labels[i], mull_a[i], mull_b[i], mull[i]))
        at = int(labels[i].split(" ")[0])
        qat_a[at] += mull_a[i]
        qat_b[at] += mull_b[i]
        qat[at] += mull[i]
        
    print("\nAtomic charges")
    print("Atom | alpha    | beta     | total    | effective")
    for i in range(len(qat)):
        print("%4d | %.6f | %.6f | %.6f | %.6f" % (i, qat_a[i], qat_b[i], qat[i], mol.atom_charge(i) - qat[i]))